In [1]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:47:18) [MSC v.1916 64 bit (AMD64)]


In [2]:
import numpy as np
np.random.seed(800)
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import tensorflow as tfa
from tensorflow import keras
os.environ['KERAS_BACKEND'] = 'tensorflow'
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [3]:
image_directory = '../data/ct_scan_dataset/'
SIZE = 512
dataset = []
label = []

In [ ]:
cancerous_images = os.listdir(image_directory + 'cancerous/')
for i, image_name in enumerate(cancerous_images):
     if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'cancerous/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [ ]:
non_cancerous_images = os.listdir(image_directory + 'non_cancerous/')
for i, image_name in enumerate(non_cancerous_images):
     if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'non_cancerous/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [ ]:
INPUT_SHAPE = (SIZE, SIZE, 3)
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size= (2,2))(conv1)
norm1 = keras.layers.BatchNormalization(axis = -1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)

conv2 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
norm2 = keras.layers.BatchNormalization(axis = -1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2)

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis = -1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)

hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis = -1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)

model = keras.Model(inputs = inp, outputs= out)
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
print(model.summary())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)

In [ ]:
train = model.fit(np.array(X_train), 
                         y_train, 
                         batch_size = 64, 
                         verbose = 1, 
                         epochs = 5,
                         validation_split = 0.1,
                         shuffle = False
                     )

In [ ]:
print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(y_test))[1]*100))